<h1>SASSIE Jet SSP Data Use and Visualization</h1>
<p> The Jet SSP is a remotely operated platform that was used to take additional measurements of physical variables near the ship track durring some parts of the campaign with fixed-depth CTD sensors and meteorlogical sensors. Jet SSP data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>
    <p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Data Download and Metadata Viewing
The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
from mpl_toolkits import mplot3d
import math
import requests

<h2>Download The Jet SSP data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Jet SSP data from PO.DAAC, which is accessed through earthdata. If you do not already have an earthdata account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your earthdata credentials below.</p>

In [ ]:
username = 'your_username'
password = 'your_password'
earthdata = requests.auth.HTTPBasicAuth(username, password)

##LOCAL DIRECTORY TO DATA
dir_in = 'Data/jetSSP/' 

The next block of code creates the directory specified above and downloads the shipboard TSG file to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
##SASSIE DATA DIR
if not os.path.exists(dir_in):
    os.makedirs(dir_in)

deployment_date = ['10','11','17','20','21','25','26']
for date in deployment_date:
    filename = 'SASSIE_Fall_2022_JetSSP_202209'+date+'.nc'
    if os.path.isfile(dir_in+filename)==False:
        url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_JET_SSP_V1/'+filename
        with requests.Session() as session:
                session.auth = (username, password)
                r1 = session.request('get', url)
                r = session.get(r1.url, auth=(username, password))
                if r.status_code == 200:
                    if r.ok:
                        with open(dir_in+filename, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + dir_in+filename) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')
    else: 
        print('JetSSP Deployment on 09/'+ date+'/2022 file is already in binder directory')

<h2> View The Metadata Inside the JetSSP Files</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 

In [ ]:
print('Displaying just 1/7 JetSSP files')
#See information about the entire dataset:
files = sorted(glob.glob(dir_in + '/*.nc'))
ds = xr.open_dataset(files[0])
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/JetSSP/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>
Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var):
    if (var=='wind_speed'): 
        colormap = 'cividis'
        var_label = 'Wind Speed (m/s)'
    if (var=='wind_direction'):
        colormap = 'twilight'
        var_label = 'Wind Direction ($^{\circ}$N)'
    if (var =='air_temperature'):
        colormap = 'magma'
        var_label = 'Air Temperature ($^{\circ}$C)'
    if (var == 'air_pressure'):
        colormap = 'Blues'
        var_label= 'Air Pressure (Pa)'
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    if var == 'time':
        colormap = 'jet'
        var_label = 'Date'
    if var=='pressure':
        colormap='Oranges'
        var_label = 'Pressure (dbar)'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>
The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin,latmax,lonmin,lonmax):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,90,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,180,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Define a Function to Create a 3D Grid of Latitude, Longitude, and Depth</h3>
The following function creates a 3D grid for plotting 3 dimensional cast data from the castaway CTD. 

In [ ]:
def make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax):    
        global fig 
        global ax
        
        fig = plt.figure(figsize=(10,8))
        ax = plt.axes(projection='3d',computed_zorder=False)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False    


        ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        xticks = list(np.linspace(lonmin,lonmax,4))
        ax.set_xticks(np.around(xticks,2))
        ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')

        yticks = list(np.linspace(latmin,latmax,4))
        ax.set_yticks(np.around(yticks,2))
        ax.invert_yaxis()
        ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

        zticks = list(np.linspace(dmin,dmax,4))
        ax.set_zticks(np.around(zticks,2))
        ax.set_zticklabels(np.around(zticks,2),ha='center',va='center', minor=False,size='small')

        ax.set_xlabel('Longitude',fontsize=13,labelpad=10,fontweight='bold')
        ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
        ax.set_zlabel('Depth (m)',fontsize=13,labelpad = 16,fontweight='bold')
        ax.view_init(195,280)
        fig.canvas.draw()
        plt.tight_layout()

<h3>Configure Supporting Data to Add to Maps</h3>
<p>The functions for viewing and plotting this data set below have options to include bathymetry, ice,  and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data, download and add AMSR Ice Data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Define a Function to Add Ice Data to a Map</h4>
This function will download the AMSR ice product at the date specified specified and add it to the plot in blue. 

In [ ]:
# DIRECTORY TO ICE DATA:
ice_dir = 'Ice_Data/'

#ICE DATA DIR
if not os.path.exists(ice_dir):
    os.makedirs(ice_dir)

def add_ice_data(ice_date):
        #get ice data at ice_date
        play_start = ice_date
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        i=0
        time_tmp=play_start - timedelta(days=i)
        while os.path.isfile(ice_dir+filename_si)==False and i<15:
            time_tmp=play_start - timedelta(days=i)
            filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(time_tmp.year)+str(time_tmp.month).zfill(2)+str(time_tmp.day).zfill(2)+'.he5'
            url = requests.get('https://n5eil01u.ecs.nsidc.org/AMSA/AU_SI12.001/'+str(time_tmp.year)+'.'+str(time_tmp.month).zfill(2)+'.'+str(time_tmp.day).zfill(2)+'/'+filename_si)
            with open(ice_dir+filename_si, 'wb') as f:
                f.write(url.content)
            i=i+1

        #apply ice data at ice_date in blue
        filename_si='AMSR_U2_L3_SeaIce12km_B04_'+str(play_start.year)+str(play_start.month).zfill(2)+str(play_start.day).zfill(2)+'.he5' 
        if os.path.isfile(ice_dir+filename_si):
            dsc = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km')
            dsd = xr.open_dataset(ice_dir+filename_si,group='HDFEOS/GRIDS/NpPolarGrid12km/Data Fields') 
            var1 = np.array(dsd.SI_12km_NH_ICECON_DAY.squeeze().values)
            indx = np.where((var1==120) | (var1==0))
            var1[indx] = 0
            ice_location = np.ma.masked_where(var1==0,var1)

            pp = ax.pcolormesh(dsc.lon,dsc.lat,ice_location, 
                            vmin=0,vmax=100, # Set max and min values for plotting
                            cmap='Blues_r', shading='auto',   # shading='auto' to avoid warning
                            transform=cartopy.crs.PlateCarree())
        

<h4>Create Directory for Shiptrack Data and Access Bathymetry Data </h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')
    
    
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code

<h2>Make Maps of This Data</h2>
<p>Using the code in this section, the user can plot the JetSSP data on a map colored by time, sea surface temperature, or sea surface salinity. The code blocks in the 'supporting code' section should be run first. The user will need to input a local directory to save figures and ice data for these to work properly. </p>

<h3> Mapping the Time and Location of JetSSP Data Collection</h3>

The map_jetSSP_tracks function shows the track of the JetSSP on a map, colored by time. Various features of this function:
    <br>1. The function automatically adds the track of the R/V Woldstad in Black for reference. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 

In [ ]:
##MAP JET SSP DEPLOYMENT LOCATIONS ON THE SHIP TRACK
def map_jetSSP_tracks(ship_track=True,bathymetry_data=True):
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS  
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-144
    
    var_label = 'Date'
    colormap = 'jet'
    
    #make the map 
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    #add a title
    ax.set_title('Jet SSP Tracks',fontsize=18) 
        
    ##OPTIONAL MAP ADD-ONs
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data == True:    
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    ##ADD JETSSP TRACKS  
    for file in files: 
        ssp_ds = xr.open_dataset(file)
        ssp_dep_lat = ssp_ds['latitude'].values
        ssp_dep_lon = ssp_ds['longitude'].values
        ssp_dep_time = ssp_ds['time'].values
        plt.legend(loc=2)
        ssp_track = ax.scatter(ssp_dep_lon,ssp_dep_lat,s=1,c=ssp_dep_time,cmap = 'jet',
                       transform=cartopy.crs.PlateCarree(),zorder=1,vmin = xr.open_dataset(files[0])['time'].values[0],vmax =xr.open_dataset(files[-1])['time'].values[-1])
    cbar = fig.colorbar(ssp_track, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label=var_label,size='large',weight='bold')
    cbar_tick_array=(np.linspace(xr.open_dataset(files[0])['time'].values[0].astype('int64'),xr.open_dataset(files[-1])['time'].values[-1].astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'deployment_map'):
        os.makedirs(fig_dir+'deployment_map')
    print('Saving Output Image:  '+fig_dir+'deployment_map/'+'SASSIE_jetSSP_deployments.png')
    plt.savefig(fig_dir+'deployment_map/'+'SASSIE_jetSSP_deployments.png',dpi='figure',format='png')
    
    

In [ ]:
map_jetSSP_tracks()

<h3>Mapping Physical Measurement Sea Surface and Atmospheric Jet SSP Data </h3>

<h4> Simple Colored Track </h4>

<p>The map_jetssp_surface_data function shows the track of the JetSSP on a map, colored by a physical measurement variable it recorded. Various features of this function: </p>
    <p>1. The user must to specify a specific deployment (there were 7 total) and physical measurement variable from the file (see xarray clickable table above) to show on the map. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>3. The user has the option to add the AMSR ice product on the date of the deployment by inputting ice_data=True.
    <br>4. The user must specify the variable to be plotted, as well as the minimum and maximum value for that variable.</p>

In [ ]:
## USE FOR SURFACE VARIABLES
def plot_jetssp_surface_data(deployment,var,var_min,var_max,ship_track=True,bathymetry_data=False,ice_data=False):
    
    #define a colormap and variable label
    colormap,var_label = define_variable_attributes(var)
    
    #GET JET SSP DATA
    ds = xr.open_dataset(files[deployment-1])
    dates = pd.to_datetime(ds.time.values).date
    deployment_date = dates[0]
    
    if (var == 'temperature') | (var=='salinity') | (var=='pressure'):
        var_arr_1 = ds[var]
        if deployment ==1:
            var_arr = var_arr_1[1,:]
        else:
            var_arr = var_arr_1[0,:]
        lat = np.squeeze(ds['latitude'])
        lon = np.squeeze(ds['longitude'])
    else:
        var_arr = ds[var]
        lat = np.squeeze(ds['latitude'])
        lon = np.squeeze(ds['longitude'])
    
    ##CREATE A MAP WITH LAND AND CITY MARKERS 
    #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
    if (deployment == 1) | (deployment == 3):
        latmin = min(lat.values)-0.01
        latmax = max(lat.values)+0.01
        lonmin = min(lon.values)-0.1
        lonmax = max(lon.values)+0.1
    else:
        latmin = min(lat.values)-0.1
        latmax = max(lat.values)+0.1
        lonmin = min(lon.values)-0.1
        lonmax = max(lon.values)+0.1
     
    #make the map:
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    #give it a title:
    ax.set_title('Jet SSP ' + str(deployment_date) +' '+ var_label +' Measurements',fontsize=15)

    
    ##OPTIONAL MAP ADD-ONs
    if ship_track==True:
        add_ship_track()
        
    if bathymetry_data == True:    
        add_bathy_data(latmin,latmax,lonmin,lonmax)
        
    if ice_data==True:
        ice_date = min(dates)
        add_ice_data(ice_date)
        
    ##APPLY JET SSP DATA TO THE MAP
    jet_ssp_track = ax.scatter(lon, 
                 lat,s=0.01, 
                 c=var_arr,cmap=colormap,
                   transform=cartopy.crs.PlateCarree(),
                      vmin = var_min,
                      vmax = var_max,zorder = 2)
    
    jet_ssp_start = ax.scatter(lon[0], 
                lat[0],s=60, 
                c='black',
                 transform=cartopy.crs.PlateCarree(),zorder = 3)
    jet_ssp_end = ax.scatter(lon[lon.size-1], 
                lat[lat.size-1],s=160, 
                c='black',marker = '*',
                 transform=cartopy.crs.PlateCarree(),zorder = 3)
                        
    matplotlib.pyplot.colorbar(jet_ssp_track, ax=ax, orientation="horizontal", pad=0.2).set_label(label=var_label,size='large',weight='bold')
    
    
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'/'+var+'/maps'):
        os.makedirs(fig_dir+'/'+var+'/maps')
        
    print('Saving Output Image:  '+fig_dir+'/'+var+'/maps/SASSIE_jetSSP_Deployment'+str(deployment)+'_'+var+'_surface.png')
    plt.savefig(fig_dir+'/'+var+'/maps/SASSIE_jet_SSP_Deployment'+str(deployment)+'_'+var+'_surface.png',dpi='figure',format='png')
    
    

In [ ]:
plot_jetssp_surface_data(4,'temperature',-0.5,2.5)
plot_jetssp_surface_data(4,'salinity',25,28)

<h4> Colored Track with Quiver Arrows </h4>

<p>The plot_wind_data function shows the track of the JetSSP on a map, colored by a physical measurement variable it recorded with quiver arrows showing the speed and direction of the wind. Various features of this function: </p>
    <p>1. The user must to specify a specific deployment (there were 7 total) and physical measurement variable from the file (see xarray clickable table above) to show on the map. 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 
    <br>4. The user must specify the variable to be plotted, as well as the minimum and maximum value for that variable.</p>

In [ ]:
##USE TO MAKE A QUIVER MAP USING JET SSP WIND SPEED AND DIRECTION DATA## 
def plot_wind_data(deployment,var,varmin,varmax,subset_res = 300,bathymetry_data=True,ship_track=True):
    if deployment==1: 
        print('No wind data taken for deployment 1')
    else:
        colormap,var_label = define_variable_attributes(var)

        #LOAD DATA FOR SELECTED DEPLOYMENT 
        ds = xr.open_dataset(files[deployment-1])
        dates = pd.to_datetime(ds.time.values).date
        deployment_date = dates[0]
        np_wdir = np.squeeze(ds['wind_direction'])
        np_wdir_adjusted = -np_wdir+90
        np_wspd = np.squeeze(ds['wind_speed'])
        np_var = np.squeeze(ds[var])[0,:]
        np_lat = np.squeeze(ds['latitude'])
        np_lon = np.squeeze(ds['longitude'])

        ##SUBSAMPLING OF VARIABLES 
        wdir = np_wdir_adjusted[0::subset_res]
        wspd = np_wspd[0::subset_res]
        var_data = np_var[0::subset_res]
        lat = np_lat[0::subset_res]
        lon = np_lon[0::subset_res]

        ##CREATE A MAP WITH LAND, CITY MARKERS, BATHYMETRY DATA(optional), AND AMSR ICE DATA AT START AND END OF PLAY(optional).
        #To show the whole campaign area, use latmin = 70, latmax =74,lonmin=-157,lonmax=-140
        if (deployment == 1) | (deployment == 3):
            latmin = min(np_lat.values)-0.01
            latmax = max(np_lat.values)+0.01
            lonmin = min(np_lon.values)-0.1
            lonmax = max(np_lon.values)+0.1
        else:
            latmin = min(np_lat.values)-0.1
            latmax = max(np_lat.values)+0.1
            lonmin = min(np_lon.values)-0.1
            lonmax = max(np_lon.values)+0.1

        #make the map: 
        map_study_area(latmin,latmax,lonmin,lonmax)
        
        #give it a title:
        ax.set_title('Jet SSP Wind Data \n'+str(deployment_date)+' Deployment',fontsize=18) 

        
        ##OPTIONAL MAP ADD ONS

        if ship_track==True:
            add_ship_track()
            
        if bathymetry_data==True: 
            add_bathy_data(latmin,latmax,lonmin,lonmax)
            
        ##APPLY WIND DATA
        u = np_wspd * np.cos(np.radians(wdir))
        v = np_wspd * np.sin(np.radians(wdir))

        p = ax.plot(np_lon.values,np_lat.values,
                    linewidth = 0.5,
                    c='black',
                    transform=cartopy.crs.PlateCarree(),label = 'Jet SSP Track',zorder=1)
        if deployment != 3:
            q = ax.quiver(lon.values,lat.values, u, v,var_data,scale = 60,width = 0.01,cmap = colormap,clim=[varmin,varmax],transform=cartopy.crs.PlateCarree())
            plt.colorbar(q, ax=ax, orientation="vertical", pad=0.1).set_label(label=var_label,size='large',weight='bold')
            
        #ADD REFERENCE ARROW
        lat_ref = latmin + abs(latmax-latmin)/8
        lon_ref = lonmax - abs(lonmax-lonmin)/4
        angle_ref = [-45+90]
        speed_ref = [10]

        u_ref = speed_ref * np.cos(np.radians(angle_ref))
        v_ref = speed_ref * np.sin(np.radians(angle_ref))

        ##SETTING REFERNCE ARROW SIZE AND LOCATION FOR EACH PLOT. 

        if deployment==2:
            text = 'Reference Arrow \n10 m/s \n45 Degrees \nClockwise of North)'
            plt.text(lon_ref-0.02,lat_ref-0.025,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
            q_ref = ax.quiver(np.array([lon_ref]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 60,width = 0.01,transform=cartopy.crs.PlateCarree())
        elif deployment==3:
            lon_ref = lonmin + abs(lonmax-lonmin)/4
            lat_ref = latmin + abs(latmax-latmin)/3
            text = 'Reference Arrow \n10 m/s \n45 Degrees \nClockwise of North)'
            plt.text(lon_ref,lat_ref-0.007,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
            q_ref = ax.quiver(np.array([lon_ref]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 150,width = 0.007,transform=cartopy.crs.PlateCarree())
            q = ax.quiver(lon.values,lat.values, u, v,var_data,scale =150,width = 0.007,cmap = colormap,clim=[varmin,varmax],transform=cartopy.crs.PlateCarree())
            plt.colorbar(q, ax=ax, orientation="vertical", pad=0.1).set_label(label=var_label,size='large',weight='bold')
        elif deployment==4: 
            lon_ref = lonmin + abs(lonmax-lonmin)/4
            lat_ref = latmin + abs(latmax-latmin)/3
            text = 'Reference Arrow \n10 m/s \n45 Degrees \nClockwise of North)'
            plt.text(lon_ref+0.04,lat_ref-0.055,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
            q_ref = ax.quiver(np.array([lon_ref]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 150,width = 0.007,transform=cartopy.crs.PlateCarree())
        elif deployment==5: 
            lon_ref = lonmin + abs(lonmax-lonmin)/4
            lat_ref = latmin + abs(latmax-latmin)/3
            text = 'Reference Arrow \n10 m/s \n45 Degrees \nClockwise of North)'
            plt.text(lon_ref+0.08,lat_ref-0.04,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
            q_ref = ax.quiver(np.array([lon_ref+0.04]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 150,width = 0.007,transform=cartopy.crs.PlateCarree())
        elif deployment==6: 
            lon_ref = lonmin + abs(lonmax-lonmin)/4
            lat_ref = latmin + abs(latmax-latmin)/3
            text = 'Reference Arrow \n10 m/s \n45 Degrees \nClockwise of North)'
            plt.text(lon_ref+0.08,lat_ref-0.04,text,transform=cartopy.crs.PlateCarree(),va='center',ha='center',multialignment='center')
            q_ref = ax.quiver(np.array([lon_ref+0.1]),np.array([lat_ref]),np.array([u_ref]),np.array([v_ref]),scale = 150,width = 0.007,transform=cartopy.crs.PlateCarree())

        plt.legend(loc=1,fontsize = 10)
        
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+'quiver_plots/'):
        os.makedirs(fig_dir+'quiver_plots/')
        
    print('Saving Output Image:  '+fig_dir+'quiver_plots/SASSIE_jetSSP_Deployment'+str(deployment)+'_'+var+'_wind.png')
    plt.savefig(fig_dir+'quiver_plots/SASSIE_jet_SSP_Deployment'+str(deployment)+'_'+var+'_wind.png',dpi='figure',format='png')
    

In [ ]:
plot_wind_data(5,'temperature',0,3)

<h2>Three Dimensional Representation of Sub-Sea Surface Trajectory Data</h2>
Use the code in this section to plot location and depth data of a selected deployment on a 3D grid, colored by a physical ocean measurement variable from the file (temperature or salinity). The user must select a deplotment (there were 7 total)

In [ ]:
## USE TO PLOT DATA VARIABLES WITH A DEPTH DIMENSION ON A 3D GRID###
def plot_jetssp_data(deployment,var):
    
    colormap,var_label = define_variable_attributes(var)

    #GET JET SSP DATA
    ds = xr.open_dataset(files[deployment-1])
    dates = pd.to_datetime(ds.time.values).date
    deployment_date = dates[0]
    var_arr = ds[var]
    lat = np.squeeze(ds['latitude'])
    lon = np.squeeze(ds['longitude'])
    depth = np.squeeze(ds['depth'])
    
    ##CREATE 3D PLOT 
    latmin = min(lat.values)
    latmax = max(lat.values)
    lonmin = min(lon.values)
    lonmax = max(lon.values)
    dmin = min(depth.values)
    dmax = max(depth.values)+0.5
    
    make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax)
    
    #give it a title
    ax.set_title('JetSSP'+ ' '+str(deployment_date)+' '+var_label,fontsize=15)
    
    #APPLY JET SSP DATA
    z_1 = np.full((1,lat.size), depth.values[0])
    z_2 = np.full((1,lat.size), depth.values[1])
    z_3 = np.full((1,lat.size), depth.values[2])
    z_4 = np.full((1,lat.size), depth.values[3])
    
    z = np.concatenate((z_1,z_2,z_3,z_4),axis=1)
    all_lon = np.concatenate((lon.values,lon.values,lon.values,lon.values), axis=0)
    all_lat = np.concatenate((lat.values,lat.values,lat.values,lat.values), axis=0)
     
    a = ax.scatter(all_lon,all_lat,z,c=(var_arr.values),cmap = colormap,s=0.5,zorder=1)
    
    fig.colorbar(a,label = var_label)
    
    if deployment == 1:
        surface = 0.29
    else:
        surface = 0.05
        
    b = ax.scatter(all_lon[0],all_lat[0],surface,c='cyan',s=40,zorder=2)
    c = ax.scatter(all_lon[all_lon.size-1],all_lat[all_lat.size-1],surface,c='cyan',s=80,zorder=2,marker='*')
    

    ##SAVE FIGURE  
    if not os.path.exists(fig_dir+var+'/profiles'):
        os.makedirs(fig_dir+var+'/profiles') 
    print('Saving Output Image:  '+fig_dir+var+'/profiles/SASSIE_jetSSP_Deployment'+str(deployment)+'_'+var+'.png')
    plt.savefig(fig_dir+var+'/profiles/SASSIE_jet_SSP_Deployment'+str(deployment)+'_'+var+'.png',dpi='figure',format='png')
    



In [ ]:
plot_jetssp_data(5,'temperature')
plot_jetssp_data(5,'salinity')

<h2>Make Timeseries Plots on a 2D Axis</h2>
The function in this section plots temperature and salinity collected from the JetSSP durring a selected Deployment on a 3 panneled time series figure.

In [ ]:
def plot_jetSSP_timeseries(deployment,var1,var2,var3): 
    
    #SECLECT AND LOAD JET SSP DATA
    ds = xr.open_dataset(files[deployment-1])
    deployment_date = pd.to_datetime(ds.time.values).date[0]
    time_data = ds['time']
    [colormap,var1_label] = define_variable_attributes(var1)
    [colormap,var2_label] = define_variable_attributes(var2)
    [colormap,var3_label] = define_variable_attributes(var3)
    var1_data = ds[var1]
    var2_data = ds[var2]
    var3_data = ds[var3]
        
    #MAKE A THREE PANELED TIME SERIES PLOT
    fig,axs = plt.subplots(3,sharex=True)
    
    if var1_data.ndim < 2:
        axs[0].plot(time_data,var1_data,linewidth=0.4,c='black')
    else:
        axs[0].plot(time_data,var1_data[0,:],linewidth=0.4,c = 'blue',label = '0.05m')
        axs[0].plot(time_data,var1_data[1,:],linewidth=0.4,c = 'red',label = '0.25m')
        axs[0].plot(time_data,var1_data[2,:],linewidth=0.4,c = 'green',label = '0.65m')
        axs[0].plot(time_data,var1_data[3,:],linewidth=0.4,c = 'cyan',label = '1.05m')
    axs[0].set(ylabel=var1_label)
    
    if var2_data.ndim<2:
        axs[1].plot(time_data,var2_data,linewidth=0.4,c='black')
    else: 
        axs[1].plot(time_data,var2_data[0,:],linewidth=0.4,c = 'blue',label = '0.05m')
        axs[1].plot(time_data,var2_data[1,:],linewidth=0.4,c='red',label = '0.25m')
        axs[1].plot(time_data,var2_data[2,:],linewidth=0.4,c='green',label = '0.65m')
        axs[1].plot(time_data,var2_data[3,:],linewidth=0.4,c='cyan',label = '1.05m')
    axs[1].set(ylabel=var2_label)
    
    if var3_data.ndim<2:
        axs[2].plot(time_data,var3_data,linewidth=0.4,c='black')
    else: 
        axs[2].plot(time_data,var3_data[0,:],c='blue',linewidth=0.4)
        axs[2].plot(time_data,var3_data[1,:],c='red',linewidth=0.4)
        axs[2].plot(time_data,var3_data[2,:],c='green',linewidth=0.4)
        axs[2].plot(time_data,var3_data[3,:],c='cyan',linewidth=0.4)
    axs[2].set(ylabel=var3_label,xlabel = 'Date')
    plt.xticks(pd.date_range(start=time_data.values[0],end=time_data.values[-1],periods=3))
    fig.suptitle(var1_label+', '+var2_label+', and '+var3_label+' Data from JetSSP '+str(deployment_date),fontsize=10)
    
    #Ideal location for legend may vary depending on which data is plotted 
    if any([var1_data.ndim>1,var2_data.ndim>1,var3_data.ndim>1]):
        #axs[1].legend(loc=1,bbox_to_anchor=(1.25,1.0))
        axs[0].legend(loc=3,ncol=4,fontsize=9)

    
        
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'/timeseries'):
        os.makedirs(fig_dir+'/timeseries')
    print('Saving Output Image:  '+fig_dir+'/timeseries/JetSSP_Deployment_'+str(deployment)+'_'+var1+'_'+var2+'_'+var3+'.png')
    plt.savefig(fig_dir+'/timeseries/JetSSP_Deployment_'+str(deployment)+'_'+var1+'_'+var2+'_'+var3+'.png',dpi='figure',format='png')
    

In [ ]:
plot_jetSSP_timeseries(4,'temperature','salinity','wind_speed')